<a href="https://colab.research.google.com/github/paulgureghian/Ludwig_Models/blob/master/Medetec_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive') 

In [0]:
import torch
from torch import nn, optim
from collections import OrderedDict
from torch.optim import lr_scheduler 
from torchvision import  models, datasets, transforms

In [0]:
data_dir = '/content/drive/My Drive/Medetec/medetec_combined_split'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [0]:
training_transforms= transforms.Compose([transforms.Resize(256),
                                         transforms.RandomRotation(45),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.RandomCrop(224),
                                         transforms.RandomResizedCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.485,0.456,0.406),
                                                                 (0.229,0.224,0.255))])

testing_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.485,0.456,0.406),
                                                              (0.229,0.224,0.255))])

training_dataset = datasets.ImageFolder(train_dir, transform=training_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform=validation_transforms)
testing_dataset = datasets.ImageFolder(test_dir, transform=testing_transforms)

training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=32)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=32) 

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = models.vgg16(pretrained=True)


for param in model.parameters():
    param.requires_grad_(False) 
    
model.classifier = nn.Sequential(OrderedDict([
                                ('fc1', nn.Linear(25088, 4096)),
                                ('relu1', nn.ReLU()),
                                ('fc2', nn.Linear(4096, 19)), 
                                ('output', nn.Softmax(dim=1))
                                ])) 

model.to(device) 
epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)  
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer) 

In [0]:
for epoch in range(epochs):
    #scheduler.step()
    train_loss = 0
    
    model.train()
    for inputs, labels in training_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        
        loss = criterion(logps, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
    model.eval()
    accuracy = 0
    valid_loss = 0
    
    for inputs, labels in validation_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        
        loss = criterion(output, labels)
        
        valid_loss += loss.item()
        
        ps = torch.exp(output)
        top_ps, top_class = ps.topk(1, dim=1) 
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    print('Epoch {}/{}.. '.format(epoch+1, epochs),
          'Training loss: {:.3f}.. '.format(train_loss / len(training_dataloader)),
          'Validation loss: {:.3f}.. '.format(valid_loss / len(validation_dataloader)),
          'Validation accuracy: {:.3f}.. '.format(accuracy / len(validation_dataloader)))
          
    model.train()      

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')
